In [ ]:
from minio import Minio
import pandas as pd
import json
from io import BytesIO

client = Minio(
    "localhost:9000",
    access_key="minio",
    secret_key="minio123",
    secure=False
)


In [10]:
# Récupération des fichiers JSON dans le bucket
objects = client.list_objects("raffinerie-raw", prefix="raw/", recursive=True)
data = []

for obj in objects:
    name = obj.object_name
    if "_spark_metadata" in name:
        continue

    try:
        response = client.get_object("raffinerie-raw", name)
        content = response.read().decode("utf-8")
        for line in content.strip().split("\n"):
            data.append(json.loads(line))
    except Exception as e:
        print(f"Erreur avec {name} : {e}")

Erreur avec raw/part-00000-a9a1d863-f1a3-4088-b31f-3f7a2ab5151b-c000.json : Expecting value: line 1 column 1 (char 0)


In [11]:
# Création DataFrame + filtrage température
df = pd.DataFrame(data)
df["timestamp"] = pd.to_datetime(df["timestamp"])
df = df.sort_values("timestamp")
df_temp = df[df["type_capteur"] == "temperature"].reset_index(drop=True)

print(f"✅ Nombre de mesures de température : {len(df_temp)}")
df_temp.to_csv("donnees_temperature.csv", index=False)
print("✅ Données de température sauvegardées dans donnees_temperature.csv")

✅ Nombre de mesures de température : 1084
✅ Données de température sauvegardées dans donnees_temperature.csv
